# Short Interest Rate Model Calibration with QuantLib-Python

## Hull-White 1 Factor Model

The Hull-White model is a practical exogenous model for fitting market interest rate term structures, described by:

$$ dr_t = (\theta(t) - a r_t) \, dt + \sigma \, dW_t $$

Where:
- \( a \) is the mean reversion constant,
- \( \sigma \) is the volatility parameter,
- \( \theta(t) \) is chosen to fit the input term structure of interest rates.

### Calibration in QuantLib-Python

To calibrate the Hull-White model in QuantLib-Python, use the `JamshidianSwaptionEngine`. This requires setting up the model with appropriate market data and then solving for the best-fit parameters \( a \) and \( \sigma \) that minimize the error in pricing known swaptions.

## Black Karasinski Model

The Black Karasinski model is an interest rate model characterized by:

$$ d \ln(r_t) = (\theta_t - a \ln(r_t)) \, dt + \sigma \, dW_t $$

### Calibration Using QuantLib-Python

As this model is non-affine, it necessitates the use of the `TreeSwaptionEngine` for calibration, which is versatile enough to handle various non-affine short rate models. The process involves fitting the model to market swaption volatilities by iteratively adjusting \( a \) and \( \sigma \).

## G2++ Model: A Two-Factor Calibration Example

The G2++ model involves two factors, \( x_t \) and \( y_t \), which add complexity and accuracy to the fitting process:

$$ dr_t = \phi(t) + x_t + y_t $$
$$ dx_t = -a x_t \, dt + \sigma \, dW^1_t \quad \text{(14.1)} $$
$$ dy_t = -b y_t \, dt + \eta \, dW^2_t \quad \text{(14.2)} $$
$$ \langle dW^1_t, dW^2_t \rangle = \rho \, dt \quad \text{(14.3)} $$

### QuantLib-Python Implementation

For calibrating the G2++ model, QuantLib-Python offers several engines including `TreeSwaptionEngine`, `G2SwaptionEngine`, and `FdG2SwaptionEngine`. The choice of engine affects both the calibration time and the accuracy of the fitted model. Calibration typically involves using historical data to estimate the parameters \( a \), \( b \), \( \sigma \), \( \eta \), and \( \rho \), ensuring the model's effectiveness in simulating and predic financial analysis and decision-making.

ry based on the choice of parameters.
le:

cribed by:





In [2]:
!pip install quantlib-python

   ---------------------------------------- 0.0/12.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.5 MB ? eta -:--:--
   ---------------------------------------- 0.1/12.5 MB 1.4 MB/s eta 0:00:09
   - -------------------------------------- 0.6/12.5 MB 4.3 MB/s eta 0:00:03
   ---- ----------------------------------- 1.4/12.5 MB 7.4 MB/s eta 0:00:02
   -------- ------------------------------- 2.5/12.5 MB 11.5 MB/s eta 0:00:01
   -------- ------------------------------- 2.6/12.5 MB 11.2 MB/s eta 0:00:01
   --------- ------------------------------ 3.0/12.5 MB 10.0 MB/s eta 0:00:01
   ------------- -------------------------- 4.2/12.5 MB 12.2 MB/s eta 0:00:01
   --------------- ------------------------ 4.7/12.5 MB 12.0 MB/s eta 0:00:01
   ------------------ --------------------- 5.7/12.5 MB 13.5 MB/s eta 0:00:01
   --------------------- ------------------ 6.7/12.5 MB 14.3 MB/s eta 0:00:01
   -------------------------- ------------- 8.4/12.5 MB 16.3 MB/s eta 0:00:01
   --

In [12]:
from QuantLib import *
from collections import namedtuple
import math
from pandas import DataFrame

In [13]:
today = Date(15, February, 2002);
settlement= Date(19, February, 2002);
Settings.instance().evaluationDate = today;
term_structure = YieldTermStructureHandle(
FlatForward(settlement,0.04875825,Actual365Fixed())
)
index = Euribor1Y(term_structure)


In [14]:
CalibrationData = namedtuple("CalibrationData",
"start, length, volatility")
data = [CalibrationData(1, 5, 0.1148),
CalibrationData(2, 4, 0.1108),
CalibrationData(3, 3, 0.1070),
CalibrationData(4, 2, 0.1021),
CalibrationData(5, 1, 0.1000 )]


In [16]:
def create_swaption_helpers(data, index, term_structure, engine):
    swaptions = []
    fixed_leg_tenor = Period(1, Years)
    fixed_leg_daycounter = Actual360()
    floating_leg_daycounter = Actual360()

    for d in data:
        vol_handle = QuoteHandle(SimpleQuote(d.volatility))
        helper = SwaptionHelper(
            Period(d.start, Years),
            Period(d.length, Years),
            vol_handle,
            index,
            fixed_leg_tenor,
            fixed_leg_daycounter,
            floating_leg_daycounter,
            term_structure
        )
        helper.setPricingEngine(engine)
        swaptions.append(helper)
        
    return swaptions


In [17]:
def calibration_report(swaptions, data):
    columns = [
        "Model Price", "Market Price", "Implied Vol", "Market Vol", 
        "Rel Error Price", "Rel Error Vols"
    ]
    report_data = []
    cum_err = 0.0
    cum_err2 = 0.0

    for i, s in enumerate(swaptions):
        model_price = s.modelValue()
        market_vol = data[i].volatility
        black_price = s.blackPrice(market_vol)
        rel_error = model_price / black_price - 1.0
        implied_vol = s.impliedVolatility(model_price, 1e-5, 50, 0.0, 0.50)
        rel_error2 = implied_vol / market_vol - 1.0
        cum_err += rel_error**2
        cum_err2 += rel_error2**2
        report_data.append(
            (model_price, black_price, implied_vol, market_vol, rel_error, rel_error2)
        )

    print("Cumulative Error Price: %7.5f" % math.sqrt(cum_err))
    print("Cumulative Error Vols : %7.5f" % math.sqrt(cum_err2))

    return DataFrame(report_data, columns=columns, index=['']*len(report_data))


In [22]:
model = HullWhite(term_structure);
engine = JamshidianSwaptionEngine(model)
swaptions = create_swaption_helpers(data, index, term_structure, engine)
optimization_method = LevenbergMarquardt(1.0e-8,1.0e-8,1.0e-8)
end_criteria = EndCriteria(10000, 100, 1e-6, 1e-8, 1e-8)
model.calibrate(swaptions, optimization_method, end_criteria)
a, sigma = model.params()
print ("a = %6.5f, sigma = %6.5f" % (a, sigma))

a = 0.04642, sigma = 0.00580


In [20]:
calibration_report(swaptions, data)

Cumulative Error Price: 0.11583
Cumulative Error Vols : 0.11614


,Model Price,Market Price,Implied Vol,Market Vol,Rel Error Price,Rel Error Vols
,0.008775,0.009485,0.106198,0.1148,-0.074854,-0.074928
,0.009669,0.010078,0.106292,0.1108,-0.040610,-0.040688
,0.008663,0.008716,0.106343,0.1070,-0.006138,-0.006138
,0.006490,0.006226,0.106442,0.1021,0.042367,0.042525
,0.003542,0.003323,0.106612,0.1000,0.065817,0.066122


In [23]:
model = BlackKarasinski(term_structure);
engine = TreeSwaptionEngine(model, 100)
swaptions = create_swaption_helpers(data, index, term_structure, engine)
optimization_method = LevenbergMarquardt(1.0e-8,1.0e-8,1.0e-8)
end_criteria = EndCriteria(10000, 100, 1e-6, 1e-8, 1e-8)
model.calibrate(swaptions, optimization_method, end_criteria)
a, sigma = model.params()
print( "a = %6.5f, sigma = %6.5f" % (a, sigma))

a = 0.03949, sigma = 0.11678


In [24]:
calibration_report(swaptions, data)

Cumulative Error Price: 0.12382
Cumulative Error Vols : 0.12415


,Model Price,Market Price,Implied Vol,Market Vol,Rel Error Price,Rel Error Vols
,0.008702,0.009485,0.105306,0.1148,-0.082623,-0.082704
,0.009669,0.010078,0.106291,0.1108,-0.040617,-0.040695
,0.008670,0.008716,0.106429,0.1070,-0.005334,-0.005334
,0.006502,0.006226,0.106636,0.1021,0.044257,0.044423
,0.003555,0.003323,0.107010,0.1000,0.069778,0.070104


In [27]:
 model = G2(term_structure);
engine = TreeSwaptionEngine(model, 25)
# engine = ql.G2SwaptionEngine(model, 10, 400)
# engine = ql.FdG2SwaptionEngine(model)
swaptions = create_swaption_helpers(data, index, term_structure, engine)
optimization_method = LevenbergMarquardt(1.0e-8,1.0e-8,1.0e-8)
end_criteria = EndCriteria(1000, 100, 1e-6, 1e-8, 1e-8)
model.calibrate(swaptions, optimization_method, end_criteria)
a, sigma, b, eta, rho = model.params()
print ("a = %6.5f, sigma = %6.5f, b = %6.5f, eta = %6.5f, rho = %6.5f " % (a, sigma, b, eta, rho))

a = 0.04758, sigma = 0.00305, b = 0.03850, eta = 0.00478, rho = -0.00000 


In [28]:
calibration_report(swaptions, data)

Cumulative Error Price: 0.12410
Cumulative Error Vols : 0.12442


,Model Price,Market Price,Implied Vol,Market Vol,Rel Error Price,Rel Error Vols
,0.008696,0.009485,0.105235,0.1148,-0.083235,-0.083316
,0.009671,0.010078,0.106320,0.1108,-0.040359,-0.040437
,0.008666,0.008716,0.106389,0.1070,-0.005712,-0.005712
,0.006500,0.006226,0.106604,0.1021,0.043950,0.044115
,0.003555,0.003323,0.107018,0.1000,0.069858,0.070184
